In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import os
path = os.getcwd()

path_mm = os.path.dirname(path)
import sys
path_mm = os.path.dirname(path_mm)
sys.path.append(path_mm)



import frequency_acquisition as Acq
#import component_acquisition

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from importlib import reload
import gc
from pyoperators import *
from pysimulators import *


from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")

# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


### Load the default dictionary and perform the changements
def get_dict(args={}):
    
    ### Get the default dictionary
    dictfilename = 'dicts/pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictfilename)
    d['npointings'] = 9
    for i in args.keys():
        
        d[str(i)] = args[i]
    
    return d

### Return frequency map according to nu and sky model
def get_pySM_maps(sky, nu):
    return np.array(sky.get_emission(nu * u.GHz, None).T * utils.bandpass_unit_conversion(nu*u.GHz, None, u.uK_CMB))

In [ ]:
Nrec = 3
Nsub = 3

if Nrec == 1:
    raise TypeError('Please use the notebook called Broadband-Reconstruction-using-PlanckData.Rmd')
elif Nsub%Nrec != 0:
    raise TypeError('Nsub/Nrec should be an integer')
    
fact_sub = int(Nsub / Nrec)
npointings = 3000
convolution = False

### Noise
noise = True
level_noise_planck = 1

### Physical band
bandpass_correction=True
band = 220
band_planck = 217
relative_bandwidth = 0.25
nside = 256
integration = 'Trapeze'

In [ ]:
reload(Acq)

d = get_dict({'npointings':npointings, 'nf_recon':Nrec, 'nf_sub':Nsub, 'nside':nside,
              'filter_nu':band * 1e9, 'noiseless': not noise,'photon_noise':noise, 'nhwp_angles':7, 'effective_duration':3, 'effective_duration':3})

sky_config = {'cmb':42, 'dust':'d0'}

qubic_acquisition = Acq.QubicIntegrated(d, Nsub=Nsub, Nrec=Nsub, integration=integration)
qubic_acquisition_recon = Acq.QubicIntegrated(d, Nsub=Nsub, Nrec=Nrec, integration=integration)
planck_acquisition = Acq.PlanckAcquisition(band_planck, qubic_acquisition.scene)
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition, planck_acquisition)

### Coverage map
cov = qubic_acquisition.get_coverage()
C_1degree = HealpixConvolutionGaussianOperator(fwhm = np.deg2rad(1))
covnorm = cov/cov.max()

### Pixels seen by QUBIC
seenpix = covnorm > 0.

In [ ]:
### CMB realization -> for real data, we should take Planck CMB map
cmb = qubic_acquisition.get_PySM_maps({'cmb':42}, np.array([150]))


### We define foregrounds model
sky = pysm3.Sky(nside, preset_strings=['d0'])
sky_model = pysm3.Sky(nside, preset_strings=['d0'])


### We compute components, we should take Planck components for real data
nu_ref = 150
plancksky = np.zeros((Nsub, 12*nside**2, 3))
skymodel = np.zeros((Nsub, 12*nside**2, 3))
C = HealpixConvolutionGaussianOperator(fwhm=np.deg2rad(0))
mysky_ref = get_pySM_maps(sky, nu_ref)
mysky_model = C(get_pySM_maps(sky_model, nu_ref))

In [ ]:
### We scale the components to reproduce frequency observations

comp = c.Dust(nu0=150)
beta = np.array([1.54, 20])
beta_model = np.array([1.54, 20])
allnus = qubic_acquisition.allnus

sed = mm.MixingMatrix(comp).evaluator(allnus)(beta)
sed_model = mm.MixingMatrix(comp).evaluator(allnus)(beta_model)

for i in range(3):
    plancksky[:, :, i] = sed @ np.array([mysky_ref[:, i]]) + cmb[0, :, i]
    skymodel[:, :, i] = sed_model @ np.array([mysky_model[:, i]])
for i in range(Nsub):
    print(f'Scaling map from {nu_ref} GHz to {qubic_acquisition.allnus[i]:.2f} GHz with factor {sed[i][0]:.2f}')
    
k=0
for i in range(Nrec):
    delta = skymodel[i*fact_sub:(i+1)*fact_sub] - np.mean(skymodel[i*fact_sub:(i+1)*fact_sub], axis=0)
    for j in range(fact_sub):
        plancksky[k] -= delta[j]
        k+=1

### We remove the bandpass mismatch for Planck data using the same model
mean_sky = np.zeros((Nrec, 12*nside**2, 3))
for i in range(Nrec):
    mean_sky[i] = np.mean(plancksky[i*fact_sub:(i+1)*fact_sub], axis=0)

In [ ]:
### Plots 
##### 1st row -> Planck sky assuming a given model to correct our reconstructed maps
##### 2nd row -> Sky model to correct the bandpass mismatch (only for astrophysical foregrounds)

plt.figure(figsize=(15, 5))

for i in range(Nsub):
    hp.mollview(plancksky[i, :, 0], min=-300, max=300, sub=(2, Nsub, i+1), cmap='jet', title=f'{qubic_acquisition.allnus[i]:.2f} GHz')
    hp.mollview(skymodel[i, :, 0], min=-300, max=300, sub=(2, Nsub, Nsub+i+1), cmap='jet', title=f'{qubic_acquisition.allnus[i]:.2f} GHz')

plt.show()

In [ ]:
### We compute QUBIC TODs with the correction of the bandpass

TOD_QUBIC = qubic_acquisition_recon.generate_tod(config=sky_config, map_ref=mysky_model, beta=beta_model, A_ev=mm.MixingMatrix(comp).evaluator(allnus), 
                                noise=noise, bandpass_correction=bandpass_correction, convolution=convolution)

In [ ]:
### We compute Planck TODs using the previous sky

TOD_PLANCK = np.zeros((Nrec, 12*nside**2, 3))
mrec = np.zeros((Nrec, 12*nside**2, 3))
n_pl = planck_acquisition.get_noise() * level_noise_planck
for irec in range(Nrec):
    
    if convolution:
        target = np.min(qubic_acquisition.allfwhm[irec*fact_sub:(irec+1)*fact_sub])
    else:
        target = 0.00
        
    C = HealpixConvolutionGaussianOperator(fwhm = target)
    mrec[irec] = C(mean_sky[irec].copy() + n_pl.copy())
    TOD_PLANCK[irec] = C(mean_sky[irec].copy() + n_pl.copy())

R = ReshapeOperator(mrec.shape, (mrec.shape[0]*mrec.shape[1]*mrec.shape[2]))
TOD_PLANCK = R(TOD_PLANCK)

In [ ]:
### Create Planck and joint acquisition

planck_acquisition = Acq.PlanckAcquisition(band_planck, qubic_acquisition_recon.scene)
qubicplanck_acquisition = Acq.QubicPlanckMultiBandAcquisition(qubic_acquisition_recon, planck_acquisition)

In [ ]:
### Create the final TOD (noise is already inside each TODs)

TOD = np.r_[TOD_QUBIC, TOD_PLANCK]

In [ ]:
### We define here the final convolution and the A and b term for PCG

myfwhm = np.array([])
for i in range(Nrec):
    myfwhm = np.append(myfwhm, np.sqrt(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]**2 - np.min(qubic_acquisition.allfwhm[i*fact_sub:(i+1)*fact_sub]**2)))
print(myfwhm)
### Reconstruction operator

H = qubicplanck_acquisition.get_operator(convolution=convolution, myfwhm=myfwhm)
beam_correction = [np.min(qubic_acquisition.allfwhm[0*fact_sub:(0+1)*fact_sub]), np.min(qubic_acquisition.allfwhm[1*fact_sub:(1+1)*fact_sub])]
invN = qubicplanck_acquisition.get_invntt_operator(beam_correction=None)

R = ReshapeOperator((1, 12*nside**2, 3), (12*nside**2, 3))

### Unpack Operator to fix some pixel during the PCG
U = (
    ReshapeOperator((Nrec * sum(seenpix) * 3), (Nrec, sum(seenpix), 3)) *
    PackOperator(np.broadcast_to(seenpix[None, :, None], (Nrec, seenpix.size, 3)).copy())
).T

### Compute A and b
with rule_manager(none=True):
    if Nrec == 1:
        A = U.T * R.T * H.T * invN * H * R * U
        x_planck = mean_sky * (1 - seenpix[None, :, None])
        b = U.T ( R.T * H.T * invN * (TOD - H(R(x_planck))))
    else:
        A = U.T * H.T * invN * H * U
        x_planck = mean_sky * (1 - seenpix[None, :, None])
        b = U.T (  H.T * invN * (TOD - H(x_planck)))
        
A = H.T * invN * H
b = H.T * invN * TOD
        
M = Acq.get_preconditioner(np.ones(12*nside**2))

In [ ]:
### PCG
solution_qubic_planck = pcg(A, b, x0=None, M=M, tol=1e-35, disp=True, maxiter=300)

In [ ]:
isub = 0
if convolution:
    target_mid_band = np.sqrt(np.mean(qubic_acquisition.allfwhm[isub*fact_sub:(isub+1)*fact_sub])**2 - np.min(qubic_acquisition.allfwhm[isub*fact_sub:(isub+1)*fact_sub])**2)
    target_min_band = np.min(qubic_acquisition.allfwhm[isub*fact_sub:(isub+1)*fact_sub])
else:
    target_mid_band = 0#np.sqrt(np.mean(qubic_acquisition.allfwhm[isub*fact_sub:(isub+1)*fact_sub])**2 - np.min(qubic_acquisition.allfwhm[isub*fact_sub:(isub+1)*fact_sub])**2)
    target_min_band = 0#np.min(qubic_acquisition.allfwhm[isub*fact_sub:(isub+1)*fact_sub])

print(target_mid_band)
print(target_min_band)

In [ ]:
C = HealpixConvolutionGaussianOperator(fwhm = target_min_band)
C1 = HealpixConvolutionGaussianOperator(fwhm = target_mid_band)
input = C1(C(plancksky[0]))

In [ ]:
plt.figure(figsize=(12, 12))

reso=15
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
stk = ['I', 'Q', 'U']

k=1
for istk in range(3):
    if istk == 0:
        min=-300
        max = 300
    else:
        min=-15
        max=15
    
    hp.gnomview(input[:, istk], rot=center, reso=reso, cmap='jet', sub=(3, 3, k), min=min, max=max, notext=True, title=f'Input - {stk[istk]}')
    hp.gnomview(C1(solution_qubic_planck['x'][isub])[:, istk], rot=center, reso=reso, cmap='jet', sub=(3, 3, k+1), min=min, max=max, notext=True, title=f'Output - {stk[istk]}')
    
    r = C1(solution_qubic_planck['x'][isub])[:, istk]-input[:, istk]
    sig = np.std(r[seenpix], axis=0)
    nsig = 2
    
    hp.gnomview(r, rot=center, reso=reso, cmap='jet', sub=(3, 3, k+2), min=-nsig*sig, max=nsig*sig, notext=True, title=f'Residual - {stk[istk]}')
    k+=3

In [ ]:
from qubic import QubicSkySim as qss

r = C1(solution_qubic_planck['x'][isub])-input

xx, yyI, yyQ, yyU = qss.get_angular_profile(np.array([r[:, 0], r[:, 1], r[:, 2]]).T, nbins=30, separate=True, center=center, thmax=60)

In [ ]:
plt.figure(figsize=(12, 8))

#plt.plot(xx, yyI, '-o', label='I')
plt.plot(xx, yyQ, '-o', label='Q')
plt.plot(xx, yyU, '-o', label='U')

plt.legend(frameon=False, fontsize=14)

plt.xlabel(r'$\theta$ [deg]', fontsize=12)
plt.ylabel(r'RMS [$\mu K^2$]', fontsize=12)
#plt.xlim(2, 50)
#plt.ylim(0, 10)

plt.show()